# Boris Integrator

In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt
import numpy as np
import pyvista as pv
import xarray as xr
from scipy import constants

import ggcmpy.tracing


def to_mesh_lines(df):
    positions = df[["x", "y", "z"]].values
    mesh = pv.PolyData(positions)
    lines = pv.lines_from_points(positions)
    return mesh, lines


def plot_trajectory(plotter, df, **kwargs):
    _, lines = to_mesh_lines(df)
    plotter.add_mesh(lines, **kwargs)

## Simple Gyration in a Uniform Magnetic Field


In [ ]:
B_0 = 1e-8  # [T]
fields = ggcmpy.tracing.UniformField(B_0=np.array([0.0, 0.0, B_0]))

q = -constants.e
m = constants.m_e
x0 = np.array([0.0, 0.0, 0.0])  # [m]
v0 = np.array([0.0, 100.0, 0.0])  # [m/s]
om_ce = np.abs(q) * B_0 / m  # [rad/s]
t_max = 2 * np.pi / om_ce  # one gyroperiod # [s]
dt = t_max / 100  # [s]

boris = ggcmpy.tracing.BorisIntegrator(fields, q, m)
df = boris.integrate(x0, v0, t_max, dt)
df

In [ ]:
df.plot(x="x", y="y", c="time", kind="scatter")
plt.axis("equal");

In [ ]:
df["E"] = 0.5 * constants.m_e * np.linalg.norm(df[["vx", "vy", "vz"]].values, axis=1)
df.plot(x="time", y="E", title="Energy of the particle over time");

## Analytical and Discretized Dipole Field

In [ ]:
field = ggcmpy.tracing.DipoleField(m=np.array([0.0, 0.0, 8e22]))  # [A m^2]

In [ ]:
R_E = 6.371e6  # [m]

x0 = np.array([5.0 * R_E, 0.0, 0.0])  # [m]
B_x0 = field.B(x0)
T_e = 1.0 * 1e3 * constants.e  # 1 keV in J
v_e = np.sqrt(2 * T_e / constants.m_e)  # electron thermal speed
v_e *= 1000.0

om_ce = np.abs(constants.e) * np.linalg.norm(B_x0) / constants.m_e  # gyrofrequency
r_ce = constants.m_e * v_e / (constants.e * np.linalg.norm(field.B(x0)))  # gyroradius

v0 = np.array([0.0, v_e, v_e])  # [m/s]
print(f"B={B_x0} om_ce={om_ce} r_ce={r_ce}")  # noqa: T201

t_max = 100.0 * 2 * np.pi / om_ce  # [s]
dt = 1.0 / om_ce / 10.0

In [ ]:
x = np.linspace(-10 * R_E, 10 * R_E, 20)
y = np.linspace(-10 * R_E, 10 * R_E, 20)
z = np.linspace(-10 * R_E, 10 * R_E, 20)
x_nc = 0.5 * (x[1:] + x[:-1])
y_nc = 0.5 * (y[1:] + y[:-1])
z_nc = 0.5 * (z[1:] + z[:-1])
coords = {"x": x, "y": y, "z": z, "x_nc": x_nc, "y_nc": y_nc, "z_nc": z_nc}

In [ ]:
b_grid = [("bx", ("x", "y", "z")), ("by", ("x", "y", "z")), ("bz", ("x", "y", "z"))]
e_grid = [("ex", ("x", "y", "z")), ("ey", ("x", "y", "z")), ("ez", ("x", "y", "z"))]

field_cc = xr.Dataset(
    ggcmpy.tracing.make_vector_field(b_grid, coords, field.B)
    | ggcmpy.tracing.make_vector_field(e_grid, coords, field.E),
    coords=coords,
)
field_cc

In [ ]:
b1_grid = [
    ("bx1", ("x_nc", "y", "z")),
    ("by1", ("x", "y_nc", "z")),
    ("bz1", ("x", "y", "z_nc")),
]
e1_grid = [
    ("ex1", ("x", "y_nc", "z_nc")),
    ("ey1", ("x_nc", "y", "z_nc")),
    ("ez1", ("x_nc", "y_nc", "z")),
]

field_yee = xr.Dataset(
    ggcmpy.tracing.make_vector_field(b1_grid, coords, field.B)
    | ggcmpy.tracing.make_vector_field(e1_grid, coords, field.E),
    coords=coords,
)

field_yee

In [ ]:
boris = ggcmpy.tracing.BorisIntegrator(field, q=-constants.e, m=constants.m_e)
df = boris.integrate(x0, v0, t_max, dt)

In [ ]:
boris_cc = ggcmpy.tracing.BorisIntegrator(field_cc, q=-constants.e, m=constants.m_e)
df_cc = boris_cc.integrate(x0, v0, t_max, dt)

In [ ]:
boris_yee = ggcmpy.tracing.BorisIntegrator(
    ggcmpy.tracing.FieldInterpolatorYee_python(field_yee),
    q=-constants.e,
    m=constants.m_e,
)
df_yee = boris_yee.integrate(x0, v0, t_max, dt)

In [ ]:
plotter = pv.Plotter()
plot_trajectory(plotter, df, line_width=1, color="blue")
plot_trajectory(plotter, df_cc, line_width=1, color="red")
plot_trajectory(plotter, df_yee, line_width=1, color="green")
plotter.show()